not finished

In [70]:
import sys, json
from datetime import datetime as dt
import numpy as np
import scipy
from preconditioners.utils import generate_c, generate_centered_gaussian_data
from preconditioners.cov_approx.variance_cov_approx import *
from sklearn.covariance import  GraphicalLasso

In [71]:
# parameters of the run
n_epochs = 100
iter_per_epoch = 500
tol = 0.00001
n = 200
d = 600
sigma2 = 1
ro = 0.5
regime = 'autoregressive'
regul_lambda = 0 # for log barrier use 0.001
lr_start = 0.5
lr_decay = 0.98

params = {
    'n_epochs' : n_epochs,
    'iter_per_epoch' : iter_per_epoch,
    'tol' : tol,
    'n' : n,
    'd' : d,
    'sigma2' : sigma2,
    'ro' : ro,
    'regime' : regime,
    'regul_lambda' : regul_lambda,
    'lr_start' : lr_start,
    'lr_decay' : lr_decay
}

In [72]:
# generate data and initialization
c = generate_c(ro=ro,
                regime=regime,
                n=n,
                d=d
                )
w_star = np.random.multivariate_normal(mean=np.zeros(d), cov=np.eye(d))
X, y, xi = generate_centered_gaussian_data(w_star,
                                            c,
                                            n=n,
                                            d=d,
                                            sigma2=sigma2,
                                            fix_norm_of_x=False)

# initialize C (cholesky), cov_inv, regul_lambda and learning rate
# is this a good way to initialize?
cov_empir = X.T.dot(X) / n
cov_inv = np.linalg.inv(cov_empir + 0.1 * np.eye(d))
C = scipy.linalg.cholesky(cov_inv) + 0.5 * generate_c(ro=0.2,
                                                        regime='autoregressive',
                                                        n=n,
                                                        d=d,
                                                        )
gl = GraphicalLasso(assume_centered=True, alpha=0.25, tol=1e-4).fit(X)
cov_gl = gl.covariance_                                    

/Users/eduardoravkin/Dropbox/My Mac (Eduard’s MacBook Pro)/Desktop/ml_research/PhD/preconditioners/src/preconditioners/utils.py:219: UserWarning: Warning, norms of datapoints are not sqrt(d)
  warnings.warn('Warning, norms of datapoints are not sqrt(d)')


In [73]:
"""
Sample code automatically generated on 2022-03-15 12:54:45

by geno from www.geno-project.org

from input

parameters
  matrix B symmetric
  matrix X
variables
  matrix C
min
  tr(inv(X*C*C'*X')*X*C*C'*B*C*C'*X'*inv(X*C*C'*X'))


The generated code is provided "as is" without warranty of any kind.
"""

from __future__ import division, print_function, absolute_import

from math import inf
from timeit import default_timer as timer
try:
    from genosolver import minimize, check_version
    USE_GENO_SOLVER = True
except ImportError:
    from scipy.optimize import minimize
    USE_GENO_SOLVER = False
    WRN = 'WARNING: GENO solver not installed. Using SciPy solver instead.\n' + \
          'Run:     pip install genosolver'
    print('*' * 63)
    print(WRN)
    print('*' * 63)



class GenoNLP:
    def __init__(self, B, X, CInit, np):
        self.np = np
        self.B = B
        self.X = X
        self.CInit = CInit
        assert isinstance(B, self.np.ndarray)
        dim = B.shape
        assert len(dim) == 2
        self.B_rows = dim[0]
        self.B_cols = dim[1]
        assert isinstance(X, self.np.ndarray)
        dim = X.shape
        assert len(dim) == 2
        self.X_rows = dim[0]
        self.X_cols = dim[1]
        assert isinstance(CInit, self.np.ndarray)
        dim = CInit.shape
        assert len(dim) == 2
        self.C_rows = dim[0]
        self.C_cols = dim[1]
        self.C_size = self.C_rows * self.C_cols
        # the following dim assertions need to hold for this problem
        assert self.B_rows == self.C_rows == self.B_cols == self.X_cols

    def getLowerBounds(self):
        bounds = []
        bounds += [-inf] * self.C_size
        return self.np.array(bounds)

    def getUpperBounds(self):
        bounds = []
        bounds += [inf] * self.C_size
        return self.np.array(bounds)

    def getStartingPoint(self):
        return self.CInit.reshape(-1)

    def variables(self, _x):
        C = _x
        C = C.reshape(self.C_rows, self.C_cols)
        return C

    def fAndG(self, _x):
        C = self.variables(_x)
        T_0 = self.np.linalg.inv((((self.X).dot(C)).dot(C.T)).dot(self.X.T))
        T_1 = (self.X.T).dot(T_0)
        T_2 = ((((T_1).dot(T_0)).dot(self.X)).dot(C)).dot(C.T)
        T_3 = ((T_2).dot(self.B.T)).dot(C)
        T_4 = (((T_1).dot(self.X)).dot(C)).dot(C.T)
        T_5 = ((T_2).dot(self.B)).dot(C)
        f_ = self.np.trace(((((((((T_0).dot(self.X)).dot(C)).dot(C.T)).dot(self.B)).dot(C)).dot(C.T)).dot(self.X.T)).dot(T_0))
        g_0 = (((((T_3 - ((((((T_3).dot(C.T)).dot(self.X.T)).dot(T_0)).dot(self.X)).dot(C) + ((((((((T_4).dot(self.B)).dot(C)).dot(C.T)).dot(self.X.T)).dot(T_0)).dot(T_0)).dot(self.X)).dot(C))) + (((((((self.B).dot(C)).dot(C.T)).dot(self.X.T)).dot(T_0)).dot(T_0)).dot(self.X)).dot(C)) + (((((((self.B.T).dot(C)).dot(C.T)).dot(self.X.T)).dot(T_0)).dot(T_0)).dot(self.X)).dot(C)) + T_5) - (((((((((T_4).dot(self.B.T)).dot(C)).dot(C.T)).dot(self.X.T)).dot(T_0)).dot(T_0)).dot(self.X)).dot(C) + (((((T_5).dot(C.T)).dot(self.X.T)).dot(T_0)).dot(self.X)).dot(C)))
        g_ = g_0.reshape(-1)
        return f_, g_

def solve(B, X, CInit, np):
    start = timer()
    NLP = GenoNLP(B, X, CInit, np)
    x0 = NLP.getStartingPoint()
    lb = NLP.getLowerBounds()
    ub = NLP.getUpperBounds()
    # These are the standard solver options, they can be omitted.
    options = {'eps_pg' : 1E-4,
               'max_iter' : 3000,
               'm' : 10,
               'ls' : 0,
               'verbose' : 10  # Set it to 0 to fully mute it.
              }

    if USE_GENO_SOLVER:
        # Check if installed GENO solver version is sufficient.
        check_version('0.1.0')
        result = minimize(NLP.fAndG, x0, lb=lb, ub=ub, options=options, np=np)
    else:
        result = minimize(NLP.fAndG, x0, jac=True, method='SLSQP',
                          bounds=list(zip(lb, ub)))

    # assemble solution and map back to original problem
    C = NLP.variables(result.x)
    elapsed = timer() - start
    print('solving took %.3f sec' % elapsed)
    return result, C

def generateRandomData(np):
    np.random.seed(0)
    B = np.random.randn(3, 3)
    B = 0.5 * (B + B.T)  # make it symmetric
    X = np.random.randn(3, 3)
    CInit = np.random.randn(3, 3)
    return B, X, CInit
import numpy as np
# import cupy as np  # uncomment this for GPU usage
print('\ngenerating random instance')
print('solving ...')

result, C = solve(B = cov_gl, X=X, CInit = C, np=np)




generating random instance
solving ...
 Iteration   FunEvals     Step Length    Function Val   Proj Gradient
         1          2               1     1.15831e+00     1.36363e-03
         2          3               1     8.44208e-01     3.26045e-04
         3          4               1     7.49832e-01     2.27151e-04
         4          5               1     6.91029e-01     3.20838e-04
         5          6               1     6.48583e-01     1.53114e-04
         6          7               1     6.39214e-01     1.36215e-04
         7          8               1     6.24823e-01     7.26546e-05
         8          9               1     6.21432e-01     9.85542e-05
         9         10               1     6.17449e-01     3.83805e-05
        10         11               1     6.15737e-01     2.46499e-05
        11         12               1     6.14062e-01     2.14058e-05
        12         13               1     6.12853e-01     3.50055e-05
        13         14               1     6.12003e

In [74]:
c

array([[1.00000000e+000, 5.00000000e-001, 2.50000000e-001, ...,
        1.92793589e-180, 9.63967946e-181, 4.81983973e-181],
       [5.00000000e-001, 1.00000000e+000, 5.00000000e-001, ...,
        3.85587178e-180, 1.92793589e-180, 9.63967946e-181],
       [2.50000000e-001, 5.00000000e-001, 1.00000000e+000, ...,
        7.71174357e-180, 3.85587178e-180, 1.92793589e-180],
       ...,
       [1.92793589e-180, 3.85587178e-180, 7.71174357e-180, ...,
        1.00000000e+000, 5.00000000e-001, 2.50000000e-001],
       [9.63967946e-181, 1.92793589e-180, 3.85587178e-180, ...,
        5.00000000e-001, 1.00000000e+000, 5.00000000e-001],
       [4.81983973e-181, 9.63967946e-181, 1.92793589e-180, ...,
        2.50000000e-001, 5.00000000e-001, 1.00000000e+000]])

In [75]:
np.linalg.inv(c)

array([[ 1.33333333e+000, -6.66666667e-001,  0.00000000e+000, ...,
         1.42695921e-196,  7.13479606e-197,  3.56739803e-197],
       [-6.66666667e-001,  1.66666667e+000, -6.66666667e-001, ...,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
       [ 0.00000000e+000, -6.66666667e-001,  1.66666667e+000, ...,
         0.00000000e+000,  0.00000000e+000,  0.00000000e+000],
       ...,
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         1.66666667e+000, -6.66666667e-001,  0.00000000e+000],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
        -6.66666667e-001,  1.66666667e+000, -6.66666667e-001],
       [ 0.00000000e+000,  0.00000000e+000,  0.00000000e+000, ...,
         0.00000000e+000, -6.66666667e-001,  1.33333333e+000]])

In [76]:
C.dot(C.T)

array([[11.07478227, -0.95379315, -0.22001497, ..., -0.02531749,
        -0.06549614,  0.02220345],
       [-0.95379315, 12.05211481, -0.56384782, ...,  0.14054591,
        -0.31947221, -0.28560944],
       [-0.22001497, -0.56384782, 11.30834085, ...,  0.21381768,
        -0.08630743,  0.01988035],
       ...,
       [-0.02531749,  0.14054591,  0.21381768, ...,  6.74194824,
        -0.15906229, -0.20055089],
       [-0.06549614, -0.31947221, -0.08630743, ..., -0.15906229,
         7.62410242,  0.41575278],
       [ 0.02220345, -0.28560944,  0.01988035, ..., -0.20055089,
         0.41575278,  6.82268864]])

In [77]:
np.linalg.inv(C.dot(C.T))

array([[ 0.14486927,  0.02841294,  0.0088292 , ...,  0.00064288,
        -0.00437891,  0.0021436 ],
       [ 0.02841294,  0.12823607,  0.02227632, ..., -0.00374597,
         0.00444023,  0.00987692],
       [ 0.0088292 ,  0.02227632,  0.1457132 , ..., -0.00663828,
         0.00636386,  0.00811664],
       ...,
       [ 0.00064288, -0.00374597, -0.00663828, ...,  0.32714795,
        -0.04527662,  0.01599243],
       [-0.00437891,  0.00444023,  0.00636386, ..., -0.04527662,
         0.34057363, -0.07437366],
       [ 0.0021436 ,  0.00987692,  0.00811664, ...,  0.01599243,
        -0.07437366,  0.36378445]])

In [78]:
cov_empir

array([[ 0.99657215,  0.45318984,  0.2292486 , ...,  0.03951084,
         0.03796301, -0.00983083],
       [ 0.45318984,  0.90459603,  0.39294534, ...,  0.05133407,
        -0.03751195, -0.08232292],
       [ 0.2292486 ,  0.39294534,  1.03292845, ...,  0.06005381,
         0.02383041, -0.01841494],
       ...,
       [ 0.03951084,  0.05133407,  0.06005381, ...,  1.05608581,
         0.42146556,  0.07348599],
       [ 0.03796301, -0.03751195,  0.02383041, ...,  0.42146556,
         0.80505165,  0.38310836],
       [-0.00983083, -0.08232292, -0.01841494, ...,  0.07348599,
         0.38310836,  0.99342184]])

In [79]:
gl = GraphicalLasso(assume_centered=True, alpha=0.25, tol=1e-4).fit(X)

In [80]:
gl.precision_

array([[ 1.05160023, -0.23620984, -0.        , ..., -0.        ,
        -0.        ,  0.        ],
       [-0.23620984,  1.18323836, -0.15640393, ..., -0.        ,
         0.        ,  0.        ],
       [-0.        , -0.15640393,  1.04905384, ..., -0.        ,
        -0.        ,  0.        ],
       ...,
       [-0.        , -0.        , -0.        , ...,  1.0356648 ,
        -0.20889973, -0.        ],
       [-0.        ,  0.        , -0.        , ..., -0.20889973,
         1.31479151, -0.17020701],
       [ 0.        ,  0.        ,  0.        , ..., -0.        ,
        -0.17020701,  1.02942771]])

In [81]:
print(np.linalg.norm(C.dot(C.T) - np.linalg.inv(c)))
print(np.linalg.norm(C.dot(C.T)/10 - np.linalg.inv(c)))
print(np.linalg.norm(gl.precision_ - np.linalg.inv(c)))
print(np.linalg.norm(np.eye(d) - np.linalg.inv(c)))

255.18284640660346
28.496966698194242
19.19023109169065
28.256759270030322


In [82]:
print(np.linalg.norm(np.diag(C.dot(C.T)/10) - np.diag(np.linalg.inv(c))))
print(np.linalg.norm(np.diag(gl.precision_) - np.diag(np.linalg.inv(c))))
print(np.linalg.norm(np.diag(np.eye(d)) - np.diag(np.linalg.inv(c))))

16.002045396212072
12.898884270794829
16.309506430300093


In [83]:
print(np.diag(gl.precision_)[:10])
print(np.diag(C.dot(C.T)-9)[:10])
print(np.diag(np.linalg.inv(c))[:10])

[1.05160023 1.18323836 1.04905384 1.05717924 1.11796397 1.43840685
 1.06769923 1.10150795 1.23079599 1.24661016]
[2.07478227 3.05211481 2.30834085 2.21405823 2.57720359 4.53697898
 1.93674428 2.25396307 2.9797089  3.47013248]
[1.33333333 1.66666667 1.66666667 1.66666667 1.66666667 1.66666667
 1.66666667 1.66666667 1.66666667 1.66666667]


In [84]:
from neurips_code.utils import *
from neurips_code.predicted_risk import *
from neurips_code.Regressor import LinearRegressor

c_e = generate_c_empir(X, empir='gl', alpha=0.25)
m = np.eye(d)
snr = 1
include_best_achievable_empirical = True
print('covariance matrix estimated')

# initialize models
reg_2 = LinearRegressor()
reg_c = LinearRegressor()
reg_ce = LinearRegressor()

# generate predictors
# matrix specifies which mirror descent we are using (GD if None)
reg_2.fit(X, y, matrix = None)
reg_c.fit(X, y, matrix = c)
reg_ce.fit(X, y, matrix = c_e)

w_a = compute_best_achievable_interpolator(X, y, c, m, snr)
reg_a = LinearRegressor(init = w_a)

if include_best_achievable_empirical:
    w_ae = compute_best_achievable_interpolator(X, y, c = c_e, m = np.eye(d), snr = 1, crossval_param = 10)
    reg_ae = LinearRegressor(init = w_ae)
    print('empirical approximation computed')

# best possible linear predictor
c_mhalf = np.linalg.inv(scipy.linalg.sqrtm(c)) # inverse square root of the covariance matrix
w_b = c_mhalf.dot( np.linalg.lstsq( X.dot(c_mhalf),  xi, rcond=None)[0] ) + w_star # best possible predictor
reg_b = LinearRegressor(init = w_b)

# calculate the expected risks
risk_2 = calculate_risk(w_star, c, reg_2.w ) + sigma2
risk_c = calculate_risk(w_star, c, reg_c.w) + sigma2
risk_ce = calculate_risk(w_star, c, reg_ce.w) + sigma2
risk_a = calculate_risk(w_star, c, reg_a.w) + sigma2
if include_best_achievable_empirical:
    risk_ae = calculate_risk(w_star, c, reg_ae.w) + sigma2
risk_b = calculate_risk(w_star, c, reg_b.w) + sigma2

risks = risk_2, risk_c, risk_ce, risk_a, risk_ae, risk_b

covariance matrix estimated
empirical approximation computed


In [85]:
risks

(322.5240568135326,
 358.5945573373417,
 329.43557275573266,
 334.01221009068973,
 326.09739570847603,
 1.5212780213237065)

In [92]:
from neurips_code.utils import *
from neurips_code.predicted_risk import *
from neurips_code.Regressor import LinearRegressor

new_P = np.linalg.inv(C.dot(C.T))
m = np.eye(d)
snr = 1
include_best_achievable_empirical = True
print('covariance matrix estimated')

# initialize models
reg_2 = LinearRegressor()
reg_c = LinearRegressor()
reg_ce = LinearRegressor()

# generate predictors
# matrix specifies which mirror descent we are using (GD if None)
reg_2.fit(X, y, matrix = None)
reg_c.fit(X, y, matrix = c)
reg_ce.fit(X, y, matrix = new_P)

w_a = compute_best_achievable_interpolator(X, y, c, m, snr)
reg_a = LinearRegressor(init = w_a)

if include_best_achievable_empirical:
    w_ae = compute_best_achievable_interpolator(X, y, c = new_P, m = np.eye(d), snr = 1, crossval_param = 10)
    reg_ae = LinearRegressor(init = w_ae)
    print('empirical approximation computed')

# best possible linear predictor
c_mhalf = np.linalg.inv(scipy.linalg.sqrtm(c)) # inverse square root of the covariance matrix
w_b = c_mhalf.dot( np.linalg.lstsq( X.dot(c_mhalf),  xi, rcond=None)[0] ) + w_star # best possible predictor
reg_b = LinearRegressor(init = w_b)

# calculate the expected risks
risk_2 = calculate_risk(w_star, c, reg_2.w ) + sigma2
risk_c = calculate_risk(w_star, c, reg_c.w) + sigma2
risk_ce = calculate_risk(w_star, c, reg_ce.w) + sigma2
risk_a = calculate_risk(w_star, c, reg_a.w) + sigma2
if include_best_achievable_empirical:
    risk_ae = calculate_risk(w_star, c, reg_ae.w) + sigma2
risk_b = calculate_risk(w_star, c, reg_b.w) + sigma2

risks = risk_2, risk_c, risk_ce, risk_a, risk_ae, risk_b

covariance matrix estimated
empirical approximation computed


In [93]:
risks

(322.5240568135326,
 358.5945573373417,
 332.4241914222392,
 334.01221009068973,
 326.9068071389576,
 1.5212780213237065)